In [9]:
def read_file(file_path):
    with open(file_path,'r',encoding='utf-8') as f:
        lines = f.readlines()
        
    row = lines[0]
    feature_name = row.strip().split('\t')[1:]
    
    row_name = []
    data = []
    for line in lines[1:]:
        row_data = line.strip().split('\t')
        row_name.append(row_data[0])
        data.append([float(i) for i in row_data[1:]])
    return feature_name,row_name,data
        
key_words,blog_titles,data = read_file('blogdata.txt')


# key_words.index('yahoo') # 3
# blog_titles.index('Quick Online Tips') # 85
# data[85][3]

In [2]:
# 构建的新的聚类结构
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
        self.left = left # 指向左子类
        self.right = right # 指向右子类
        self.vec = vec # 上文的data,即 词的频率
        self.id = id # 文档的ID
        self.distance = distance # 左子类和右子类的相似度

In [3]:
from math import sqrt
def pearson(v1,v2):
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    sum1sq = sum([pow(v,2) for v in v1])
    sum2sq = sum([pow(v,2) for v in v2])
    
    multi_sum = sum([v1[i] * v2[i] for i in range(len(v1))])
    
    num = multi_sum - (sum1 * sum2 / len(v1))
    den = sqrt((sum1sq - pow(sum1, 2) / len(v1)) * (sum2sq - pow(sum2,2) / len(v2)))
    if den == 0:
        return 0
    return 1.0 - num/den

In [4]:
'''
聚类函数 核心
算法核心是将文档两两pearson值计算，将两个距离最小的合并成一个新类，同时利用树的结构保存原来的关系，直至只有一个大类，算法结束
'''
def hcluster(rows,distance=pearson):
    # 初始时每个数据都是一类，用列表存储
    clust = [bicluster(rows[i],id=i) for i in range(len(rows))]
    
    distance_pair = {}
    currentclustid=-1 # -1 表示构建生成的类ID
    # 得到树根（即一个类）时退出循环
    while len(clust) > 1:
        lowestpair = (0,1)
        closest = distance(clust[0].vec, clust[1].vec)
        # 两两对比，寻找最小距离
        for i in range(len(clust)):
            for j in range(i+1, len(clust)):
                if (clust[i].id, clust[j].id) not in distance_pair:
                    distance_pair[(clust[i].id, clust[j].id)] = distance(clust[i].vec, clust[j].vec)
                
                d = distance_pair[(clust[i].id, clust[j].id)]
                if d < closest:
                    closest = d
                    lowestpair = (i,j)
        
        # 计算两个最小距离对应的类的平均值
        mergevec=[(clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 for i in range(len(clust[0].vec))]
        # 将上面两个类合并生成新的类，左右分支分别指向原来的类
        newcluster=bicluster(mergevec,left=clust[lowestpair[0]],right=clust[lowestpair[1]],distance=closest,id=currentclustid)
        
        currentclustid-=1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)

    return clust[0],distance_pair

result,result1 = hcluster(data)

In [5]:
# 字符版打印聚类，理解成树
def printclust(clust,labels=None,n=0):
    for i in range(n): 
        print(' ',end="")
    if clust.id<0:
        print('-')
    else:
        if labels==None:
            print(clust.id)
        else:
            print(labels[clust.id])

    if clust.left!=None:
        printclust(clust.left,labels=labels,n=n+1)
    if clust.right!=None:
        printclust(clust.right,labels=labels,n=n+1)
        
printclust(result,blog_titles)

-
 gapingvoid: "cartoons drawn on the back of business cards"
 -
  -
   Schneier on Security
   Instapundit.com
  -
   The Blotter
   -
    -
     MetaFilter
     -
      SpikedHumor
      -
       Captain's Quarters
       -
        Michelle Malkin
        -
         -
          NewsBusters.org - Exposing Liberal Media Bias
          -
           -
            Hot Air
            Crooks and Liars
           -
            Power Line
            Think Progress
         -
          Andrew Sullivan | The Daily Dish
          -
           Little Green Footballs
           -
            Eschaton
            -
             Talking Points Memo: by Joshua Micah Marshall
             Daily Kos
    -
     43 Folders
     -
      TechEBlog
      -
       -
        Mashable!
        Signum sine tinnitu--by Guy Kawasaki
       -
        -
         -
          Slashdot
          -
           MAKE Magazine
           Boing Boing
         -
          -
           Oilman
           -
            Online

In [6]:
from PIL import Image,ImageDraw


In [7]:
import random

def k_means_cluster(rows,distance=pearson,k=4):
    # 计算每个维度(特征值)的最小值和最大值，即每一列的最小值和最大值
    ranges=[(min([row[i] for row in rows]),max([row[i] for row in rows])) for i in range(len(rows[0]))]
    
    # 随机创建K个质心
    k_center = [[random.random() * ( ranges[i][1] - ranges[i][0]) + ranges[i][0] for i in range(len(rows[0]))] for j in range(k)]
    
    lastmatches = None
    for t in range(100):
        # 保存到质心距离最短的最佳文档集
        bestmatches = [[] for i in range(k)]
        
        # 针对每个文档的归属聚类判断开始
        # 遍历每一篇文档
        for j in range(len(rows)):
            # 取得当前文档
            row = rows[j]
            # 初始化当前文档的最佳聚类ID
            bestmatch = 0
            # 遍历每一个质心
            for i in range(k):
                # 计算当前文档与当前质心的距离
                dis = distance(row,k_center[i])
                # 如果得到的距离小于当前文档与原来最佳质心的距离，则更新最佳的聚类质心
                if dis < distance(row, k_center[bestmatch]):
                    bestmatch = i
            # 将聚类质心ID和文档ID关系出处起来
            bestmatches[bestmatch].append(j)
        # 针对每个文档的归属聚类判断结束
        
        # 提前结束
        if bestmatches == lastmatches:
            break
        lastmatches = bestmatches
        
        # 更新类簇质心开始
        # 遍历每一个类簇
        for i in range(k):
            # 初始化一个列表用来存储更新后的质心坐标
            avgs = [0.0] * len(rows[0])

            # 如果当前类簇的文档数量大于0
            if len(bestmatches[i]) > 0:
                # 遍历当前类簇的文档ID集合
                for rowid in bestmatches[i]:
                    # 计算文档集合中每个维度的总和
                    for m in range(len(rows[rowid])):
                        avgs[m] += rows[rowid][m]
                # 计算每个维度的均值，每个维度的总和/当前类簇的文档数量
                for j in range(len(avgs)):
                    avgs[j] /= len(bestmatches[i])
                # 更新当前类簇为最新的均值
                k_center[i] = avgs
                    
    return bestmatches

k_means_cluster(data,k=3)

[[2,
  5,
  7,
  13,
  16,
  21,
  23,
  24,
  25,
  30,
  34,
  36,
  40,
  48,
  49,
  56,
  63,
  65,
  67,
  68,
  69,
  79,
  80,
  84,
  85,
  91,
  94],
 [4, 41, 42, 45, 46, 81],
 [0,
  1,
  3,
  6,
  8,
  9,
  10,
  11,
  12,
  14,
  15,
  17,
  18,
  19,
  20,
  22,
  26,
  27,
  28,
  29,
  31,
  32,
  33,
  35,
  37,
  38,
  39,
  43,
  44,
  47,
  50,
  51,
  52,
  53,
  54,
  55,
  57,
  58,
  59,
  60,
  61,
  62,
  64,
  66,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  82,
  83,
  86,
  87,
  88,
  89,
  90,
  92,
  93,
  95,
  96,
  97,
  98]]

In [8]:
'''
tanimoto 系数
表示两个向量中的交集与并集的比值
v1和v2是相同维度的向量
return:
返回一个介于0和1之间的数
其中1表示每个维度都被两个向量所共有
0表示两个向量之间没有交集
'''
def tanimoto(v1,v2):
    c1,c2,shr = 0,0,0
    for i in range(len(v1)):
        if v1[i] != 0:
            c1 += 1
        if v2[i] != 0:
            c2 += 1
        if v1[i] != 0 and v2[i] != 0:
            shr += 1
    return float(shr)/(c1+c2-shr)
# 但是，根据距离函数的意义，即两个向量距离越近，返回的值应该越小，所以改写上述代码
def tanimote(v1,v2):
    c1,c2,shr = 0,0,0
    for i in range(len(v1)):
        if v1[i] != 0:
            c1 += 1
        if v2[i] != 0:
            c2 += 1
        if v1[i] != 0 and v2[i] != 0:
            shr += 1
    # 用1减去比率并返回
    # 此时，1表示两个向量之间没有交集，距离比较远
    # 0表示两个向量交集 == 并集，距离比较近
    return 1-float(shr)/(c1+c2-shr)

In [20]:
user,item,data = read_file('zebo.txt')

In [13]:
result,result1 = hcluster(data)

In [21]:
printclust(result,item)

-
 -
  -
   -
    -
     dog
     -
      cell phone
      -
       horse
       playstation 3
    -
     -
      xbox 360
      ps3
     -
      phone
      -
       food
       computer
   -
    mansion
    -
     -
      family
      friends
     -
      boyfriend
      -
       money
       -
        job
        -
         -
          clothes
          shoes
         -
          dvd player
          -
           cds
           -
            psp
            tv
  -
   love
   puppy
 -
  house
  -
   house and lot
   -
    -
     <b>car</b>
     -
      jeans
      -
       bike
       mobile
    -
     -
      -
       mp3 player
       digital camera
      -
       laptop
       ipod
     -
      watch
      cellphone
